# RABI

In [1]:
%load_ext autoreload
%autoreload 2
from FIFOv4 import FIFO
import matplotlib.pyplot as plt
import numpy as np
import math
from spcm import units
import spcm as spcm
import time as time
from time import sleep 
# import API classes into the current namespace
from pulsestreamer import PulseStreamer, Sequence  
import hdawg_driver_latest_v2 as awg
from sg396_driver_v1 import SG396
from pulser import PulseGenerator
from laser_driver import LaserControl
from thorlabs_laser_shutter_driver import LaserShutter
from typing import List


### Connect Instruments

In [ ]:
# Digitizer 
FIFO = FIFO('dev/spcm0')

In [2]:
# Pulse Streamer 
PSch_MW=2
PSch_Digitizer=1 
PSch_Laser= 3
PS_chmap={'mw':PSch_MW,
          'digtrig':PSch_Digitizer,
          'laser':PSch_Laser}
ip = '10.135.70.193' 
ps = PulseGenerator(ip,PS_chmap)
INF = np.iinfo(np.int64).max


Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 10.135.70.193


AssertionError: No Pulse Streamer found at IP/Host-address: 10.135.70.193

In [ ]:
# AWB
awg = awg.HDAWG('dev8181', '127.0.0.1', 8004)

In [ ]:
# Signal Generator 
sg = SG396('TCPIP::10.135.70.65::inst0::INSTR')

In [6]:
laser = LaserControl('LAS-08166')
laser_shutter = LaserShutter('68800950')

### Configure Settings

In [3]:
def choose_sideband(opt, nv_freq, side_freq, pulse_axis='x'):
    match pulse_axis:
        case 'y':
            delta = 90
        case _: 
            delta = 0

    match opt:
        case 'Upper':
            frequencies = nv_freq - side_freq
            iq_phases = [delta+90, delta+0]
        case 'Both':
            frequencies = nv_freq - side_freq
            iq_phases = [delta+0, delta+90, delta+90, delta+0] # lower sideband phases + upper sideband phases
        case _:
            frequencies = nv_freq + side_freq
            iq_phases = [delta+0, delta+90]

    return frequencies, iq_phases

In [2]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive

# Define the parameters (as provided)
awg_parameters = {
    "seq": "RABI",
    "i_offset": -2*1e-3, # mV
    "q_offset": -4*1e-3, # mV
    "sideband_power": 150 * 1e-3, # mV
    "sideband_freqs": None, # MHz
    "iq_phases": [0, 90],
    "num_pts": 40,
    "runs": 1000,
    'start': 0,
    'stop': 10,
    'mw_times': None,
    'sig_gen_freq': None,
    'pulse_axis': None,
    'sideband': None,
    'freq': None,
    'rf_power' : None}

dig_config = {
    'num_pts_in_exp': 2 * awg_parameters['num_pts'], # MW ON + OFF subsequences
    'runs': awg_parameters['runs'], # number of exp. iterations
    'segment_size': 512 * 10,
    'pretrig_size': 32,
    'readout_ch': int(0),
    'HF_INPUT_50OHM': 1
}

# Calculate sideband frequencies
mw_times = np.linspace(awg_parameters['start'], awg_parameters['stop'], awg_parameters['num_pts']) * 1e9

awg_parameters['mw_times'] = mw_times


In [ ]:
match awg_parameters['array_type']:
                case 'geomspace':
                    tau_times = np.geomspace(awg_parameters['start'], awg_parameters['stop'], awg_parameters['num_pts']) * 1e9
                case 'linspace':
                    tau_times = np.linspace(awg_parameters['start'], awg_parameters['stop'], awg_parameters['num_pts']) * 1e9


In [ ]:
sig_gen_freq, iq_phases = choose_sideband(awg_parameters['sideband'], awg_parameters['freq'], awg_parameters['sideband_freqs'], awg_parameters['pulse_axis'])
awg_parameters['sig_gen_freq'] = sig_gen_freq
awg_parameters['iq_phases'] = iq_phases

In [ ]:
pi: List[float] = []
pi_half: List[float] = []

for i in range(2):
            pi_half.append(kwargs['pi']*1e9/2)
            pi.append(kwargs['pi']*1e9)




In [ ]:
sig_gen.set_frequency(awg_parameters['sig_gen_freq']) # set carrier frequency
sig_gen.set_rf_amplitude(awg_parameters['rf_power']) # set MW power
sig_gen.set_mod_type(7) # quadrature amplitude modulation
sig_gen.set_mod_subtype(1) # no constellation mapping
sig_gen.set_mod_function('IQ', 5) # external modulation
sig_gen.set_mod_toggle(1) # turn on modulation mode

In [ ]:
# Create interactive widgets to display the values
def display_awg_parameters():
    display(widgets.HTML(value=f"<b>Averaging Sequence: </b>{awg_parameters['seq']}"))
    display(widgets.HTML(value=f"<b>i_offset: </b>{awg_parameters['i_offset']} mV"))
    display(widgets.HTML(value=f"<b>q_offset: </b>{awg_parameters['q_offset']} mV"))
    display(widgets.HTML(value=f"<b>sideband_power: </b>{awg_parameters['sideband_power']} mV"))
    display(widgets.HTML(value=f"<b>iq_phases: </b>{awg_parameters['iq_phases']}"))
    display(widgets.HTML(value=f"<b>num_pts: </b>{awg_parameters['num_pts']}"))
    display(widgets.HTML(value=f"<b>sideband_freqs: </b>{awg_parameters['sideband_freqs']} MHz"))
    display(widgets.HTML(value=f"<b>runs: </b>{awg_parameters['runs']}"))

    display(widgets.HTML(value="<br><b>Digitizer Configuration:</b>"))
    display(widgets.HTML(value=f"<b>num_pts_in_exp: </b>{dig_config['num_pts_in_exp']}"))
    display(widgets.HTML(value=f"<b>segment_size: </b>{dig_config['segment_size']}"))
    display(widgets.HTML(value=f"<b>pretrig_size: </b>{dig_config['pretrig_size']}"))
    display(widgets.HTML(value=f"<b>readout_ch: </b>{dig_config['readout_ch']}"))
    display(widgets.HTML(value=f"<b>HF_INPUT_50OHM: </b>{dig_config['HF_INPUT_50OHM']}"))

# Display parameters
display_awg_parameters()

### Assign Configuration 

In [ ]:
awg.set_sequence(**{'seq': 'T2',
                'seq_dd': awg_parameters['t2_seq'],
                'i_offset': awg_parameters['i_offset'],
                'q_offset': awg_parameters['q_offset'],
                'sideband_power': awg_parameters['sideband_power'],
                'sideband_freq': awg_parameters['sideband_freq'], 
                'iq_phases': iq_phases,
                'pihalf_x': pi_half[0]/1e9,
                'pihalf_y': pi_half[1]/1e9,
                'pi_x': pi[0]/1e9, 
                'pi_y': pi[1]/1e9,
                'n': awg_parameters['n'],
                'num_pts': awg_parameters['num_pts'],
                'runs': awg_parameters['runs'], 
                'iters': awg_parameters['iters']}) 

In [ ]:
FIFO.assign_param(settings_dict=dig_config)
FIFO.config()

### Configure ODMR Pulse Sequence 

In [12]:
import typing as t

In [20]:
_T = t.TypeVar('_T')

def convert_type(arg: t.Any, converter: _T) -> _T:
    return converter(arg)

In [21]:
readout_time = 1000
trig_time = 10
initial_delay = 1
singlet_decay = 1
laser_time = 1
trig_spot = 1
clock_time = 1
MW_buffer_time = 1
awg_trig_time = 1


def Echo(params, pihalf_x, pihalf_y, pi_x, pi_y):
    '''
    Spin Echo pulse sequence.
    MW sequence: pi/2(x) - tau - pi(y) - tau - pi/2(x)
    
    '''
    longest_time =  convert_type(round(max(params)), float)
    pihalf_x =  convert_type(round(pihalf_x), float)
    pihalf_y =  convert_type(round(pihalf_y), float)
    pi_x =  convert_type(round(pi_x), float)
    pi_y =  convert_type(round(pi_y), float)

    def SingleEcho(tau_time):
        '''
        CREATE SINGLE HAHN-ECHO SEQUENCE TO REPEAT THROUGHOUT EXPERIMENT
        '''

        tau_time =  convert_type(round(tau_time), float) # convert to proper data type to avoid undesired rpyc netref data type
        
        '''
        DEFINE SPECIAL TIME INTERVALS FOR EXPERIMENT
        '''
        # padding time to equalize duration of every run (for different tau durations)
        pad_time = longest_time - tau_time

        '''
        DEFINE RELEVANT ON, OFF TIMES FOR DEVICES
        '''            
        laser_off1 =  initial_delay
        laser_off2 =  singlet_decay + pihalf_x + tau_time + pi_y + tau_time + pihalf_x +  MW_buffer_time
        # laser_off2 =  singlet_decay + pihalf_x + tau_time + pi_y + tau_time + pihalf_x +  MW_buffer_time
        laser_off3 = 100 + pad_time 

        # digitizer trigger timing
        clock_off1 = laser_off1 +  laser_time + laser_off2 +  trig_spot -  clock_time
        clock_off2 = -  trig_spot +  readout_time + laser_off3

        # mw I & Q off windows (on slightly longer than VSG to ensure it's set)
        iq_off1 = laser_off1 +  laser_time +  singlet_decay
        iq_off2 = (pihalf_x -  awg_trig_time) + tau_time
        iq_off3 = (pi_y -  awg_trig_time) + tau_time
        iq_off4 = (pihalf_x -  awg_trig_time) +  MW_buffer_time +  readout_time + laser_off3

        '''
        CONSTRUCT PULSE SEQUENCE
        '''

        # define sequence structure for laser
        laser_seq = [(laser_off1, 0), ( laser_time, 1), (laser_off2, 0), ( readout_time, 1), (laser_off3, 0)]
        laser_seq_tot = laser_seq + laser_seq
        # define sequence structure for digitizer trigger
        dig_clock_seq = [(clock_off1, 0), ( clock_time, 1), (clock_off2, 0)]
        dig_clock_seq_tot = dig_clock_seq + dig_clock_seq
        # define sequence structure for MW I and Q when MW = ON
        mw_iq_seq = [(iq_off1, 0), ( awg_trig_time, 1), (iq_off2, 0), ( awg_trig_time, 1), (iq_off3, 0), ( awg_trig_time, 1), (iq_off4, 0)]
        mw_iq_seq_tot = mw_iq_seq + mw_iq_seq
        # assign sequences to respective channels for seq_on

        return [laser_seq_tot, dig_clock_seq_tot, mw_iq_seq_tot]    
    
    laser_ps_tot=[]
    dig_ps_tot=[]
    mw_ps_tot=[]
    for tau in params:
        seqs += SingleEcho(tau)
        laser_ps_tot += Echo(tau)[0]
        dig_ps_tot += Echo(tau)[1]
        mw_ps_tot += Echo(tau)[2]

    return [laser_ps_tot, dig_ps_tot, mw_ps_tot]


In [3]:
sequence = Echo(tau_times, pi_half[0], pi_half[1], 
                                pi[0], pi[1])
ps.setDigital("laser", sequence[0]) # digitizer trigger
ps.setDigital("digtrig", sequence[1]) # digitizer trigger
ps.setDigital("mw", sequence[2]) # MW IQ
ps.setTrigger
ps.plotSeq(plot_all=False)

NameError: name 'Rabi' is not defined

In [ ]:
FIFO.config()
FIFO.start_buffer()
laser_shutter.open_shutter()
laser.set_modulation_state('pulsed')
laser.set_analog_control_mode('current')
laser.set_diode_current_realtime(awg_parameters['laser_power'])
laser.laser_on()


In [ ]:
sg.set_rf_toggle(1)
ps.stream(n_runs=INF)
DATA=FIFO.acquire()

In [ ]:
FIFO.stop_card()
FIFO.reset()

In [142]:
# copy=[]
# for i in range(0,10):
#     FIFO.config()
#     sg.set_rf_toggle(1)
#     FIFO.start_buffer()
#     DATA=FIFO.acquire()
#     FIFO.stop_card()
#     FIFO.reset()
#     sg.set_rf_toggle(0)
#     copy.append(DATA)

# LASER OFF

In [167]:
#STOP 
ps.reset()
sg.set_rf_toggle(0)
awg.set_disabled()
FIFO.reset()

### Data Post Processing

In [162]:
np.shape(DATA)
import pandas as pd

In [ ]:
reshaped_data = DATA.squeeze(axis=2)  # Reduces (20000, 5120, 1) to (20000, 5120)
data_to_save = pd.DataFrame(reshaped_data)

In [145]:
DATA_mean=np.mean(DATA,axis=1)
segments=(np.shape(DATA)[0])


In [146]:
sig=DATA_mean[::2]
bg=DATA_mean[1::2]

In [148]:
sig_mean=np.zeros(awg_parameters['num_pts'])
bg_mean=np.zeros(awg_parameters['num_pts'])

In [ ]:
ms1_array = np.ones(awg_parameters['num_pts']) # dark data array
ms0_array = np.ones(awg_parameters['num_pts']) # bright data array

for i in range(awg_parameters['num_pts']):
    ms1_array[i] = np.mean(sig_mean[i::awg_parameters['num_pts']])
    ms0_array[i] = np.mean(bg_mean[i::awg_parameters['num_pts']])
    

In [149]:
#MAKE PI PULSE TIME 


In [ ]:
plt.plot(awg_parameters['mw_times'],sig_mean)
plt.plot(awg_parameters['mw_times'],bg_mean)
